# Explorative Notebook
To see how code functions

## Geo-FNO

In [2]:
import torch
import numpy as np

In [6]:
inputX = np.load(r'C:\Users\Noahc\Documents\USYD\PHD\8 - Github\Rev909\data\NACA_Cylinder_X.npy')
inputY = np.load(r'C:\Users\Noahc\Documents\USYD\PHD\8 - Github\Rev909\data\NACA_Cylinder_Y.npy')

In [7]:
print(inputX.shape, inputY.shape)

(2490, 221, 51) (2490, 221, 51)


In [8]:
inputX = torch.tensor(inputX, dtype=torch.float)
inputY = torch.tensor(inputY, dtype=torch.float)
input = torch.stack([inputX, inputY], dim=-1)

In [9]:
input.shape

torch.Size([2490, 221, 51, 2])

In [11]:
inputX[0,:-1,0] - inputX[0,1:,0]

tensor([ 1.5097e+01,  9.1194e+00,  5.5086e+00,  3.3275e+00,  2.0100e+00,
         1.2141e+00,  7.3341e-01,  4.4302e-01,  2.6761e-01,  1.6165e-01,
         9.7644e-02,  5.8982e-02,  3.5628e-02,  3.4930e-02,  3.4245e-02,
         3.3574e-02,  3.2915e-02,  3.2270e-02,  3.1637e-02,  3.1017e-02,
         3.0409e-02,  2.9812e-02,  2.9228e-02,  2.8655e-02,  2.8093e-02,
         2.7542e-02,  2.7002e-02,  2.6472e-02,  2.5953e-02,  2.5445e-02,
         2.4945e-02,  2.4457e-02,  2.3977e-02,  2.3507e-02,  2.3046e-02,
         2.2594e-02,  2.2151e-02,  2.1717e-02,  2.1291e-02,  2.0873e-02,
         2.0464e-02,  2.0063e-02,  1.9669e-02,  1.9284e-02,  1.8906e-02,
         1.8535e-02,  1.8172e-02,  1.7815e-02,  1.7466e-02,  1.7123e-02,
         1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6949e-02,
         1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6949e-02,
         1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6949e-02,
         1.6949e-02,  1.6949e-02,  1.6949e-02,  1.6

In [13]:
import plotly.graph_objects as go

In [22]:
plt = go.Figure()
for i in range(51):
    plt.add_trace(go.Scatter(x=inputX[0,:,i], y=inputY[0,:,i],  mode='markers', marker={'size':2}))
plt.show()

**Geo-FNO datasets are still `stuctured`**. We can interpolate a dataset onto contructed ring of data for the cylinder flow case. But other geometries are a bit difficult.

## Soblev Norms

In [3]:
# load in sample data
dataset = np.load(r"C:\Users\Noahc\Documents\USYD\PHD\0 - Work Space\Markov Studies\2D_NS_Re40.npy")
print(dataset.shape)

(200, 501, 64, 64)


In [4]:
sample_solution = dataset[[0],200,:,:]
x = torch.tensor(sample_solution)
y = torch.tensor(sample_solution)

In [5]:
# soblev 1st
k = 1
balanced = False
size_average = False
a = None
if a == None:
    #a = [1,] * k
    a = [1,1]

In [6]:
nx = x.size()[1]
ny = x.size()[2]
#k = self.k
#balanced = self.balanced
#a = self.a
x = x.view(x.shape[0], nx, ny, -1)
y = y.view(y.shape[0], nx, ny, -1)

k_x = torch.cat((torch.arange(start=0, end=nx//2, step=1),torch.arange(start=-nx//2, end=0, step=1)), 0).reshape(nx,1).repeat(1,ny)
k_y = torch.cat((torch.arange(start=0, end=ny//2, step=1),torch.arange(start=-ny//2, end=0, step=1)), 0).reshape(1,ny).repeat(nx,1)
k_x = torch.abs(k_x).reshape(1,nx,ny,1).to(x.device)
k_y = torch.abs(k_y).reshape(1,nx,ny,1).to(x.device)

x_hat = torch.fft.fftn(x, dim=[1, 2])
y_hat = torch.fft.fftn(y, dim=[1, 2])

if balanced==False:
    weight = 1
    component_k1 = a[0]**2 * (k_x**2 + k_y**2)
    component_k2 = a[1]**2 * (k_x**4 + 2*k_x**2*k_y**2 + k_y**4)
    if k >= 1:
        weight += component_k1
    if k >= 2:
        weight += component_k2
    weight = torch.sqrt(weight)
    #loss = self.rel(x*weight, y*weight)
else:
    raise NotImplementedError
    loss = self.rel(x, y)
    if k >= 1:
        weight = a[0] * torch.sqrt(k_x**2 + k_y**2)
        loss += self.rel(x*weight, y*weight)
    if k >= 2:
        weight = a[1] * torch.sqrt(k_x**4 + 2*k_x**2*k_y**2 + k_y**4)
        loss += self.rel(x*weight, y*weight)
    loss = loss / (k+1)
 
# Note: loss is computed in fourier space

In [7]:
first_soblev = component_k1*x_hat
first_soblev_inverse = torch.fft.ifftn(first_soblev, dim=[1, 2]).real
first_soblev.shape

torch.Size([1, 64, 64, 1])

In [8]:
print('x', x.shape)
print('x_hat', x_hat.shape)
print('first_soblev', first_soblev.shape)
print('first_soblev_inverse', torch.fft.ifftn(first_soblev, dim=[1, 2]).real.shape)

x torch.Size([1, 64, 64, 1])
x_hat torch.Size([1, 64, 64, 1])
first_soblev torch.Size([1, 64, 64, 1])
first_soblev_inverse torch.Size([1, 64, 64, 1])


In [9]:
import plotly.graph_objects as go
import plotly.subplots as subplots
rows=1
cols=4
fig = subplots.make_subplots(rows,cols)
fig.add_trace(go.Heatmap(z=x[0,...,0], showscale=False),1,1)
fig.add_trace(go.Heatmap(z=x_hat[0,...,0].real, showscale=False),1,2)
fig.add_trace(go.Heatmap(z=first_soblev[0,...,0].real, showscale=False),1,3)
fig.add_trace(go.Heatmap(z=first_soblev_inverse[0,...,0], showscale=False),1,4)
for i in range(1, rows*cols + 1):
    fig.update_xaxes(scaleanchor=f"y{i}", scaleratio=1, row=(i-1)//cols + 1, col=(i-1)%cols + 1)
    fig.update_yaxes(constrain='domain', row=(i-1)//cols + 1, col=(i-1)%cols + 1)
fig.show()

According to ChatGPT, these codes refer to the Laplacian and the bi-Laplacian each.

In [10]:
# def periodic_derivatives(x, dx=1.0, dy=1.0):
#     """
#     Compute first and second derivatives with 2nd-order central differences
#     and periodic BCs.
    
#     x: tensor of shape (B, N, N) or (B, N, N, C)
#     dx, dy: grid spacing
    
#     Returns:
#         dx1, dy1 : first derivatives
#         dx2, dy2 : second derivatives
#     """

#     # shift along x-direction (dim=1)
#     x_ip = torch.roll(x, shifts=-1, dims=1)
#     x_im = torch.roll(x, shifts=+1, dims=1)

#     # shift along y-direction (dim=2)
#     y_ip = torch.roll(x, shifts=-1, dims=2)
#     y_im = torch.roll(x, shifts=+1, dims=2)

#     # First derivatives: (f[i+1] - f[i-1]) / (2*dx)
#     dx1 = (x_ip - x_im) / (2.0 * dx)
#     dy1 = (y_ip - y_im) / (2.0 * dy)

#     # Second derivatives: (f[i+1] - 2f[i] + f[i-1]) / dx^2
#     dx2 = (x_ip - 2.0 * x + x_im) / (dx * dx)
#     dy2 = (y_ip - 2.0 * x + y_im) / (dy * dy)

#     return dx1, dy1, dx2, dy2

In [11]:
from my_utils import periodic_derivatives

In [12]:
dx1, dy1, dx2, dy2 = periodic_derivatives(x, dx=np.pi/32, dy=np.pi/32)

In [13]:
H1_sq = (x**2 + dx1**2 + dy1**2)#.sum(dim=[1,2])

In [14]:
rows=1
cols=4
fig = subplots.make_subplots(rows,cols)
fig.add_trace(go.Heatmap(z=x[0,...,0], showscale=False),1,1)
#fig.add_trace(go.Heatmap(z=H1_sq[0,...,0], showscale=False),1,2)
fig.add_trace(go.Heatmap(z=-(dx2+dy2)[0,...,0], showscale=False),1,2)
fig.add_trace(go.Heatmap(z=(first_soblev_inverse+(dx2+dy2))[0,...,0], showscale=False),1,3)
fig.add_trace(go.Heatmap(z=first_soblev_inverse[0,...,0], showscale=False),1,4)
for i in range(1, rows*cols + 1):
    fig.update_xaxes(scaleanchor=f"y{i}", scaleratio=1, row=(i-1)//cols + 1, col=(i-1)%cols + 1)
    fig.update_yaxes(constrain='domain', row=(i-1)//cols + 1, col=(i-1)%cols + 1)
fig.show()

In [15]:
inverse_weight = torch.fft.ifftn(weight, dim=[1, 2]).real

In [16]:
rows=1
cols=4
fig = subplots.make_subplots(rows,cols)
fig.add_trace(go.Heatmap(z=x[0,...,0], showscale=False),1,1)
fig.add_trace(go.Heatmap(z=H1_sq[0,...,0], showscale=False),1,2)
fig.add_trace(go.Heatmap(z=-(dx2+dy2)[0,...,0], showscale=False),1,3)
fig.add_trace(go.Heatmap(z=inverse_weight[0,...,0], showscale=False),1,4)
for i in range(1, rows*cols + 1):
    fig.update_xaxes(scaleanchor=f"y{i}", scaleratio=1, row=(i-1)//cols + 1, col=(i-1)%cols + 1)
    fig.update_yaxes(constrain='domain', row=(i-1)//cols + 1, col=(i-1)%cols + 1)
fig.show()

**Validation Study**

- Train with and without Sobolev Norms, confirm better performance
- Validate that a FDM Sobolev Norm performs similarly
- Only then consider FVM

## Solution analysis

In [31]:
from utilities import PCA

from plotly.subplots import make_subplots

In [76]:
model_1 = np.load(r'C:\Users\Noahc\Documents\USYD\PHD\0 - Work Space\Markov Studies v2\NS_fourier_MNO_dissipative_N_180_k0_gTrue_ep50_m20_w64.npy') # poor regularizer
model_2 = np.load(r'C:\Users\Noahc\Documents\USYD\PHD\0 - Work Space\Markov Studies v2\NS_fourier_MNO_dissipative_N_180_k1_gTrue_ep50_m20_w64.npy') # good regularizer
model_3 = np.load(r'C:\Users\Noahc\Documents\USYD\PHD\0 - Work Space\Markov Studies v2\NS_fourier_MNO_dissipative_N_180_k1_gTrue_ep50_m20_w64_FDM.npy')
truth   = np.load(r'C:\Users\Noahc\Documents\USYD\PHD\0 - Work Space\Markov Studies v2\2D_NS_Re500_sample.npy')
truth_t, truth_b, S, _ = truth.shape
model_1 = torch.tensor(model_1).permute(2,0,1)
model_2 = torch.tensor(model_2).permute(2,0,1)
model_3 = torch.tensor(model_3).permute(2,0,1)
truth = torch.tensor(truth).reshape(truth_t*truth_b, S, S)
T = truth.shape[0]
n_T = model_1.shape[0]
print(model_1.shape, model_2.shape, truth.shape)

torch.Size([10000, 64, 64]) torch.Size([10000, 64, 64]) torch.Size([2005, 64, 64])


In [77]:
PCA_dim = 50
x_pca = PCA(truth.reshape(-1, 64*64), PCA_dim, subtract_mean=False)
pca_truth = x_pca.encode(truth.reshape(-1, 64*64))[:T]
pca_model_1 = x_pca.encode(model_1.reshape(-1, 64*64))[:T]
pca_model_2 = x_pca.encode(model_2.reshape(-1, 64*64))[:T]
pca_model_3 = x_pca.encode(model_3.reshape(-1, 64*64))[:T]

In [78]:
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scatter3d"}, {"type": "scatter"}]])

fig.add_trace(go.Scatter3d(x=pca_truth[:n_T, 0], y=pca_truth[:n_T, 1], z=np.arange(n_T), mode='markers', marker=dict(size=1, color='blue'), name= 'Dataset', legendgroup='Dataset'), row=1, col=1)
fig.add_trace(go.Scatter3d(x=pca_model_1[:n_T, 0],  y=pca_model_1[:n_T, 1],  z=np.arange(n_T), mode='markers', marker=dict(size=1, color='red'),  name= 'H0 reg',  legendgroup='H0 reg'), row=1, col=1)
fig.add_trace(go.Scatter3d(x=pca_model_2[:n_T, 0],  y=pca_model_2[:n_T, 1],  z=np.arange(n_T), mode='markers', marker=dict(size=1, color='green'),  name= 'H1 reg',  legendgroup='H1 reg'), row=1, col=1)
fig.add_trace(go.Scatter3d(x=pca_model_3[:n_T, 0],  y=pca_model_3[:n_T, 1],  z=np.arange(n_T), mode='markers', marker=dict(size=1, color='purple'),  name= 'H1 reg FDM',  legendgroup='H1 reg FDM'), row=1, col=1)
fig.update_layout(width=600, height=600, scene=dict(xaxis_title=r'λ1', yaxis_title=r'λ2', zaxis_title='Time-step'))

fig.add_trace(go.Scatter(x=pca_truth[:n_T, 0], y=pca_truth[:n_T, 1], mode='markers', marker=dict(size=2, color='blue'), name= 'Dataset', legendgroup='Dataset', showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=pca_model_1[:n_T, 0],  y=pca_model_1[:n_T, 1],  mode='markers', marker=dict(size=2, color='red'),  name= 'H0 reg',  legendgroup='H0 reg',  showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=pca_model_2[:n_T, 0],  y=pca_model_2[:n_T, 1],  mode='markers', marker=dict(size=2, color='green'),  name= 'H1 reg',  legendgroup='H1 reg',  showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=pca_model_3[:n_T, 0],  y=pca_model_3[:n_T, 1],  mode='markers', marker=dict(size=2, color='purple'),  name= 'H1 reg FDM',  legendgroup='H1 reg FDM',  showlegend=False), row=1, col=2)
fig.update_layout(width=1200, height=600, scene=dict(xaxis_title=r'λ1', yaxis_title=r'λ2'), xaxis=dict(scaleanchor='y'),yaxis=dict(scaleratio=1))
#fig.update_xaxes(range=[-100, 100])
#fig.update_yaxes(range=[-100, 100])
fig.show()

In [79]:
def spectrum2(u, s):
    T = u.shape[0]
    u = u.reshape(T, s, s)
    u = torch.fft.fft2(u)

    # 2d wavenumbers following Pytorch fft convention
    k_max = s // 2
    wavenumers = torch.cat((torch.arange(start=0, end=k_max, step=1), \
                            torch.arange(start=-k_max, end=0, step=1)), 0).repeat(s, 1)
    k_x = wavenumers.transpose(0, 1)
    k_y = wavenumers
    
    # Sum wavenumbers
    sum_k = torch.abs(k_x) + torch.abs(k_y)
    sum_k = sum_k.numpy()
    
    # Remove symmetric components from wavenumbers
    index = -1.0 * np.ones((s, s))
    index[0:k_max + 1, 0:k_max + 1] = sum_k[0:k_max + 1, 0:k_max + 1]

    spectrum = np.zeros((T, s))
    for j in range(1, s + 1):
        ind = np.where(index == j)
        spectrum[:, j - 1] = np.sqrt( (u[:, ind[0], ind[1]].sum(axis=1)).abs() ** 2)
        
    spectrum = spectrum.mean(axis=0)
    return spectrum

In [80]:
spec_truth = spectrum2(truth, S)[:S//2]
spec_model_1 = spectrum2(model_1, S)[:S//2]
spec_model_2 = spectrum2(model_2, S)[:S//2]
spec_model_3 = spectrum2(model_3, S)[:S//2]

In [82]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=spec_truth, name='Dataset', line={'color':'Black'}))
fig.add_trace(go.Scatter(y=spec_model_1, name='H0'))
fig.add_trace(go.Scatter(y=spec_model_2, name='H1'))
fig.add_trace(go.Scatter(y=spec_model_3, name='H1 FDM'))
fig.update_layout(
    yaxis=dict(type="log"),
    width=800, height=600
)
fig.show()

In [112]:
from scipy.stats import gaussian_kde
target_data_data = truth.reshape(-1)[np.random.permutation(len(truth.reshape(-1)))[:10000]]
target_data_m1 = model_1.reshape(-1)[np.random.permutation(len(model_1.reshape(-1)))[:10000]]
target_data_m2 = model_2.reshape(-1)[np.random.permutation(len(model_2.reshape(-1)))[:10000]]
target_data_m3 = model_3.reshape(-1)[np.random.permutation(len(model_3.reshape(-1)))[:10000]]
kde_data = gaussian_kde(target_data_data, bw_method='scott')
kde_m1 = gaussian_kde(target_data_m1, bw_method='scott')
kde_m2 = gaussian_kde(target_data_m2, bw_method='scott')
kde_m3 = gaussian_kde(target_data_m3, bw_method='scott')
bins_data = np.linspace(target_data_data.min(), target_data_data.max(), 50)
bins_m1 = np.linspace(target_data_m1.min(), target_data_m1.max(), 50)
bins_m2 = np.linspace(target_data_m2.min(), target_data_m2.max(), 50)
bins_m3 = np.linspace(target_data_m3.min(), target_data_m3.max(), 50)
density_data = kde_data.evaluate(bins_data)
density_m1 = kde_m1.evaluate(bins_m1)
density_m2 = kde_m2.evaluate(bins_m2)
density_m3 = kde_m3.evaluate(bins_m3)

In [106]:
truth.shape

torch.Size([2005, 64, 64])

In [110]:
torch.mean(truth,dim=(1,2)).max()

tensor(0.0034, dtype=torch.float64)

In [113]:
fig = make_subplots(rows=1, cols=2, column_titles=('Vorticity over Time','Vorticity Distribution'))
fig.add_trace(go.Scatter(x=np.arange(100, 100+target_data_m1.shape[0]),  y=torch.mean(model_1,dim=(1,2)),  mode='lines', line=dict(color='Teal'), showlegend=False,  name= 'H0',  legendgroup='H0'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(100, 100+target_data_m2.shape[0]),  y=torch.mean(model_2,dim=(1,2)),  mode='lines', line=dict(color='Purple'), showlegend=False,  name= 'H1',  legendgroup='H1'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(100, 100+target_data_m3.shape[0]),  y=torch.mean(model_3,dim=(1,2)),  mode='lines', line=dict(color='Red'), showlegend=False,  name= 'H1 FDM',  legendgroup='H1 FDM'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(100, 100+target_data_data.shape[0]),  y=torch.mean(truth,dim=(1,2)),  mode='lines', line=dict(color='Grey'), showlegend=False,  name= 'Data',  legendgroup='Data'), row=1, col=1)
fig.add_trace(go.Scatter(x=bins_data, y=density_data, mode='lines', line=dict(color='Grey'),  name= 'Data',  legendgroup='Data'), row=1, col=2)
fig.add_trace(go.Scatter(x=bins_m1, y=density_m1, mode='lines', line=dict(color='Teal'),  name= 'H0',  legendgroup='H0'), row=1, col=2)
fig.add_trace(go.Scatter(x=bins_m2, y=density_m2, mode='lines', line=dict(color='Purple'),  name= 'H1',  legendgroup='H1'), row=1, col=2)
fig.add_trace(go.Scatter(x=bins_m3, y=density_m3, mode='lines', line=dict(color='Red'),  name= 'H1 FDM',  legendgroup='H1 FDM'), row=1, col=2)
fig.update_xaxes(title_text="time (s)", row=1, col=1); fig.update_yaxes(title_text="Mean Vorticity", row=1, col=1)
fig.update_xaxes(title_text="Vorticity", row=1, col=2); fig.update_yaxes(title_text="density", row=1, col=2)
fig.show()